In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder, StandardScaler, RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from loguru import logger
from datetime import datetime
import uuid

In [16]:
class FeatureEngineeringProcessor:
    def __init__(self, raw_data: pd.DataFrame, pipeline_name: str) -> None:
        # Guarda el DataFrame original.
        self.raw_data = raw_data
        # Guarda el nombre del pipeline.
        self.pipeline_name = pipeline_name
        # Inicializa la tabla de características como None.
        self.feature_table = None

    def impute_scale(self, n_components: int = 2) -> pd.DataFrame:
        # Define las columnas numéricas a procesar.
        numeric_cols= [
            "lead_time",
            "adults",
            "children",
            "babies",
            "adr"
        ]
        pipe = Pipeline(
            steps=[
                # Imputa valores faltantes con la media.
                ("imputer_mean", SimpleImputer(strategy="mean")),
                # Escala las variables numéricas.
                ("std_scaling", StandardScaler()),
                # Reduce la dimensionalidad con PCA.
                ("pca", PCA(n_components=n_components))
            ]
        )
        # Devuelve un DataFrame con las nuevas características numéricas.
        return pd.DataFrame(
            pipe.fit_transform(self.raw_data[numeric_cols]),
            columns=["great_feature1", "great_feature2"]
        )

    def encode_categoricals(self) -> pd.DataFrame:
        encoded_vars = []
        for var in ["hotel", "market_segment", "reserved_room_type"]:
            # Muestra en el log qué variable se está codificando.
            logger.info(f"Codificando con OHE {var}")
            encoder = OneHotEncoder()
            # Codifica la variable categórica usando OneHotEncoder.
            encoded = encoder.fit_transform(self.raw_data[[var]]).toarray()
            cols  = [f"{var}_{col}" for col in encoder.categories_[0]]
            # Genera los nombres de las columnas codificadas.
            _dataframe = pd.DataFrame(
                encoded,
                columns= cols
            )
            # Añade el DataFrame codificado a la lista.
            encoded_vars.append(_dataframe)
        # Devuelve la concatenación de todos los DataFrames codificados.
        return pd.concat(encoded_vars,axis=1)

    def run(self) -> pd.DataFrame:
        # Log de inicio del pipeline.
        logger.info(f"Inicializando pipeline {self.pipeline_name}")

        # Codifica las variables categóricas.
        categorical = self.encode_categoricals()
        # Procesa las variables numéricas.
        numerics = self.impute_scale()

        # Une las variables categóricas y numéricas.
        modeling_dataset = pd.concat([categorical, numerics], axis=1)

        pipe = Pipeline(
            steps=[
                # Elimina variables con baja varianza.
                ("feature_selection", VarianceThreshold()),
                # Escala las variables usando RobustScaler.
                ("scaling_robust", RobustScaler())
            ]
        )
        # Aplica el pipeline y guarda el resultado en feature_table.
        self.feature_table =  pd.DataFrame(
            pipe.fit_transform(modeling_dataset),
            columns=modeling_dataset.columns
        )

        # Añade una columna de IDs únicos.
        self.feature_table["booking_id"] = [str(uuid.uuid4()) for _ in range(self.feature_table.shape[0])]
        # Añade una columna de timestamp.
        self.feature_table["event_timestamp"] = [datetime.now() for _ in range(self.feature_table.shape[0])]
        
        import time
        # Espera 1 segundo.
        time.sleep(1)
        # Añade una columna de timestamp de creación.
        self.feature_table["created"] = [datetime.now() for _ in range(self.feature_table.shape[0])]

        # Devuelve la tabla final de características.
        return self.feature_table

    def write_feature_table(self, filepath: str) -> None:
        # Log de escritura de la tabla.
        logger.info(f"Escribiendo feature table en {filepath}")
        if not self.feature_table.empty: # -> True o False
            # Guarda la tabla en formato parquet.
            self.feature_table.to_parquet(f"{filepath}.parquet", index=False)
            # Guarda la tabla en formato csv.
            self.feature_table.to_csv(f"{filepath}.csv", index=False)
        else:
            # Lanza excepción si la tabla no existe.
            raise Exception("La feature table no ha sido creada. Ejecutar el comando .run()")  

In [6]:
# Dataset Hotel Booking -> https://www.kaggle.com/datasets/jessemostipak/hotel-booking-demand/data
raw_data = pd.read_csv("../data/raw/hotel_bookings.csv")

# Train Test Split

In [7]:
# Divide el DataFrame en conjuntos de entrenamiento y prueba usando scikit-learn.
train_raw_data, test_raw_data = train_test_split(
    raw_data, test_size=0.2, random_state=42
)

In [8]:
train_raw_data["reserved_room_type"].value_counts()

reserved_room_type
A    68710
D    15375
E     5243
F     2314
G     1695
B      914
C      762
H      483
P       11
L        5
Name: count, dtype: int64

## OneHotEncoder

In [10]:
train_raw_data["hotel"].value_counts()

hotel
City Hotel      63486
Resort Hotel    32026
Name: count, dtype: int64

In [13]:

# Crea una instancia del codificador OneHotEncoder.
encoder = OneHotEncoder()

# Ajusta el codificador a la columna 'hotel' del conjunto de entrenamiento y transforma los datos en una matriz codificada (one-hot).
encoded = encoder.fit_transform(train_raw_data[["hotel"]])

# Muestra las categorías únicas encontradas en la columna 'hotel' durante el ajuste.
encoder.categories_[0]

array(['City Hotel', 'Resort Hotel'], dtype=object)

In [15]:
train_processor = FeatureEngineeringProcessor(
    raw_data=train_raw_data,
    pipeline_name="train_pipeline"
)
# Ejecuta el procesamiento de características.
train_processor.run()

# Guarda la tabla de características en disco.
train_processor.write_feature_table("../data/processed/bookings_feature_table")

[array(['City Hotel', 'Resort Hotel'], dtype=object)]

In [ ]:
test_processor = FeatureEngineeringProcessor(
    raw_data=test_raw_data,
    pipeline_name="test_pipeline"
)
# Ejecuta el procesamiento de características.
test_processor.run()